In [177]:
import pandas as pd
import datetime
import numpy as np
import pytz
import time
import csv
import importlib

In [3]:
#load in the metadata dataframe 
colNames = ["epTitle", "epDescription", "duration", "pubDate", "copyright", "itunes:type", 
           "itunes:complete", "guid", "itunes:explicit", "enclosure","itunes:image", "transDict"]

#this *appears* to be loading correctly 
META_PATH= "/shared/3/projects/benlitterer/podcastData/podRss/floydFeeds/floydMetadata.csv"
metaDf = pd.read_csv(META_PATH, names=colNames).reset_index(names=["rssUrl"])

/tmp/ipykernel_2867304/1307880365.py:7: DtypeWarning: Columns (5,6,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  metaDf = pd.read_csv(META_PATH, names=colNames).reset_index(names=["rssUrl"])


In [4]:
POD_PATH = "/shared/3/projects/benlitterer/podcastData/podcastIndex/floydSubset.csv"
podDf = pd.read_csv(POD_PATH, index_col=0).drop(columns=["Unnamed: 0"]).rename(columns={"url":"rssUrl", "description":"podDescription"})

/tmp/ipykernel_2867304/174181308.py:2: DtypeWarning: Columns (1,8,9,14,24,25,38) have mixed types. Specify dtype option on import or set low_memory=False.
  podDf = pd.read_csv(POD_PATH, index_col=0).drop(columns=["Unnamed: 0"]).rename(columns={"url":"rssUrl", "description":"podDescription"})


In [183]:
LOG_PATH = "/home/blitt/projects/podcasts/mergeTransMetadata/logs/mergeTransMetadata/Jan2_2024.txt"
logFile = open(LOG_PATH, "w") 
logFile.write(f"shape of RSS scraped metadata: {metaDf.shape}\n")
logFile.write(f"shape of podcastIndex metadata: {podDf.shape}\n")
logFile.flush()

In [5]:
print(podDf.shape)
print(podDf.dropna(subset=["link"]).shape)

(2115280, 41)
(2085463, 41)


In [8]:
#merge the metadata onto the podcast episode data 
merged = pd.merge(metaDf, podDf, on=["rssUrl"], how="left")

In [ ]:
logFile.write(f"shape of data merged on rssUrl: {merged.shape}\n") 
logFile.flush()

In [9]:
merged.dropna(subset=["epTitle"]).shape

(65715102, 53)

In [10]:
len(metaDf) == len(merged) 

True

In [73]:
def parseTime(inStr, inFormat):
    try: 
        return datetime.datetime.strptime(inStr, inFormat)

        return pd.to_datetime(inStr, format=inFormat).tz_localize(None)
    except Exception as E:
        #print(E)
        return np.nan

In [118]:
samp = merged.head(1000000)

In [114]:
pd.to_datetime(testStr, format="%a, %d %b %Y %H:%M:%S %z").tz_localize(None)

Timestamp('2023-05-01 08:00:00')

In [120]:
(65 * 18) / 60

19.5

In [121]:
sTime = time.time()
samp['date_formats_with_offset'] = samp["pubDate"].apply(parseTime, inFormat="%a, %d %b %Y %H:%M:%S %z")
samp["date_formats_without_offset"] = samp["pubDate"].apply(parseTime, inFormat="%a, %d %b %Y %H:%M:%S %Z") 
samp["cleanDates"] = samp['date_formats_with_offset'].fillna(samp["date_formats_without_offset"])
print(time.time() - sTime)

/tmp/ipykernel_2867304/348279712.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samp['date_formats_with_offset'] = samp["pubDate"].apply(parseTime, inFormat="%a, %d %b %Y %H:%M:%S %z")
/tmp/ipykernel_2867304/348279712.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samp["date_formats_without_offset"] = samp["pubDate"].apply(parseTime, inFormat="%a, %d %b %Y %H:%M:%S %Z")


19.354294776916504


/tmp/ipykernel_2867304/348279712.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samp["cleanDates"] = samp['date_formats_with_offset'].fillna(samp["date_formats_without_offset"])


In [123]:
#get just the english first to save time
merged = merged.dropna(subset=["language"])
logFile.write(f"shape of data without NA's in language column: {merged.shape}\n") 

merged = merged[merged["language"].str.lower().str.contains('en')]
logFile.write(f"shape of data with only English: {merged.shape}\n") 


In [125]:
# clean date using only one format, leaving NaTs for unmatched format 
# tz_localize(None) means that our times are local to the places that they originated 
merged['date_formats_with_offset'] = merged["pubDate"].apply(parseTime, inFormat="%a, %d %b %Y %H:%M:%S %z")
merged["date_formats_without_offset"] = merged["pubDate"].apply(parseTime, inFormat="%a, %d %b %Y %H:%M:%S %Z") 
merged["cleanDates"] = merged['date_formats_with_offset'].fillna(merged["date_formats_without_offset"])

In [ ]:
logFile.write(f"finished date formatting\n") 

In [126]:
naDates = sum(merged["cleanDates"].isna())
logFile.write(f"# of date NA values: {naDates}\n")
logFile.flush()

0

In [179]:
#other cleaning things to do... 
#add transcript path 
#get function to give transcript file path of urls
spec=importlib.util.spec_from_file_location("getURLstorageLocation","/home/blitt/projects/podcasts/mergeTransMetadata/getURLstorageLocation.py")
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)

#get path to transcripts for each url 
#we will have to add META_PATH later, since we could be sending output to different plcaes 
META_PATH= ""
merged = merged.dropna(subset=["enclosure"])
logFile.write(f"shape of data after removing NA urls: {merged.shape}\n") 
merged["potentialOutPath"] = merged["enclosure"].apply(foo.getUrlTranscriptPath, args=[META_PATH])
logFile.flush()

52

In [128]:
merged = merged.dropna(subset=["cleanDates"]) 
logFile.write(f"shape after removing cleanDate NA values: {merged.shape}\n") 

In [129]:
#we started with 300k na's and ended at ~700k 
#so we have ~ 400k na values out of ~27 million rows 

In [130]:
merged = merged.drop(columns=['date_formats_with_offset', 'date_formats_without_offset'])

In [83]:
#merged.head(300)["cleanDates"].apply(lambda x: x.replace(tzinfo=None))  > datetime.datetime.strptime("03-04-2023", "%m-%d-%Y") 

In [ ]:
#merged = merged.rename(columns={"description_x":"epDescription", "description_y":"podDescription"})

In [ ]:
merged["cleanDatesLoc"] = pd.to_datetime(merged["cleanDates"], utc=True, errors="coerce")
logFile.write(f"forcing clean dates to UTC format (no offset assumes +00:00)\n") 

In [ ]:
logFile.write(f"writing merged file to disk\n") 
merged.to_csv("/shared/3/projects/benlitterer/podcastData/processed/mergedMetaDataClean.csv", quoting=csv.QUOTE_NONNUMERIC)
logFile.flush()

In [171]:
#before cutoff
firstCutoff = pd.to_datetime("05-03-2020", format="%m-%d-%Y", utc=True)
lastCutoff = pd.to_datetime("05-09-2020", format="%m-%d-%Y", utc=True)


In [152]:
#merged.sample(30)[["pubDate", "cleanDates"]]

In [172]:
#punch in here
beforeFMonth = merged[(merged["cleanDatesLoc"] >= firstCutoff) & (merged["cleanDatesLoc"] <= lastCutoff)]

In [185]:
#punch in here 
#logFile.write(f"dates for beforeFMonth are [05-03-2020, 05-09-2020], inclusive\n") 
#logFile.write(f"writing week before Floyd Month file to disk\n") 
#logFile.flush()
beforeFMonth.to_csv("/shared/3/projects/benlitterer/podcastData/processed/beforeFloydMonth/beforeFMonth.tsv", quoting=csv.QUOTE_NONNUMERIC)

In [145]:
#two weeks before was Monday, May 11, 2020
#two weeks after was Monday, June 8, 2020
logFile.write(f"dates for Floyd Month are [05-11-2020, 06-08-2020], inclusive\n") 
logFile.write(f"writing Floyd Month file to disk\n") 
logFile.flush()

firstCutoff = pd.to_datetime("05-11-2020", format="%m-%d-%Y", utc=True)
lastCutoff = pd.to_datetime("06-08-2020", format="%m-%d-%Y", utc=True)

#OLD Line that was here 
#floydMonths = merged[(merged["date_format_same_final"] >= "05-11-2020") & (merged["date_format_same_final"] <= "06-08-2020")]

floydMonths = merged[(merged["cleanDatesLoc"] >= firstCutoff) & (merged["cleanDatesLoc"] <= lastCutoff)]

In [151]:
floydMonths.to_csv("/shared/3/projects/benlitterer/podcastData/processed/floydMonth/floydMonthEnSHORT.csv", quoting=csv.QUOTE_NONNUMERIC)
logFile.write("finished") 
logFile.close()